In [1]:
from eval import strip_gloss_punctuation
def read_st_file(path):
    all_glosses = []

    with open(path, 'r') as infile:
        for line in infile:
            line_prefix = line[:2]
            if line_prefix == '\\g':
                preds = line[3:].strip()
                preds = strip_gloss_punctuation(preds)
                all_glosses.append(preds)
    return all_glosses

len(read_st_file('../preds/2023glossingSTsubmissions/SigMoreFun_2/arp-test-track2-covered.txt'))

/Users/milesper/.pyenv/versions/3.10.0/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/milesper/.pyenv/versions/3.10.0/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/milesper/.pyenv/versions/3.10.0/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


4892

In [2]:
import datasets
dataset = datasets.load_dataset('lecslab/glosslm-split')

In [4]:
import pandas as pd
import os

iso_to_glotto = {
    'arp': 'arap1274',
    "git": "gitx1241",
    "lez": "lezg1247",
    "ntu": "natu1246",
    "nyb": "nyan1302",
    "ddo": "dido1241",
    "usp": "uspa1245",
}

def create_df(preds, isocode, segmented):
    assert isocode in iso_to_glotto
    glottocode = iso_to_glotto[isocode]
    id_or_ood = "ID" if glottocode in ["arap1274", "dido1241", "uspa1245"] else "OOD"
    gold = dataset[f'test_{id_or_ood}'].filter(lambda row: row['glottocode'] == glottocode and row['is_segmented'] == segmented)

    assert len(gold) == len(preds), f"Length mismatch: {len(gold)} expected, {len(preds)} observed"

    gold_glosses = [strip_gloss_punctuation(g) for g in gold["glosses"]]
    return pd.DataFrame({
        "id": gold["id"],
        "glottocode": gold["glottocode"],
        "is_segmented": gold["is_segmented"],
        "pred": preds,
        "gold": gold_glosses,
    }), id_or_ood

def process_folder(dirpath):
    data = {}

    for file in os.listdir(dirpath):
        if not file.endswith('txt'):
            continue
        filename = os.fsdecode(file)
        path = os.path.join(dirpath, filename)
        comps = filename.split("-")
        preds = read_st_file(path)
        df, id_or_ood = create_df(preds, comps[0], 'yes' if comps[2] == 'track2' else 'no')
        if comps[0] in data:
            data[comps[0]] = pd.concat([data[comps[0]], df])
        else:
            data[comps[0]] = df

    id = pd.concat([data['arp'], data['ddo'], data['usp']])
    id.to_csv(os.path.join(dirpath, 'test_ID-preds.csv'))
    ood = pd.concat([data['git'], data['lez'], data['ntu'], data['nyb']])
    ood.to_csv(os.path.join(dirpath, 'test_OOD-preds.csv'))

process_folder('../preds/2023glossingSTsubmissions/mt5_odin_punct_fix')

Filter:   0%|          | 0/972 [00:00<?, ? examples/s]

Filter:   0%|          | 0/11940 [00:00<?, ? examples/s]

Filter:   0%|          | 0/972 [00:00<?, ? examples/s]

Filter:   0%|          | 0/972 [00:00<?, ? examples/s]

Filter:   0%|          | 0/972 [00:00<?, ? examples/s]

Filter:   0%|          | 0/11940 [00:00<?, ? examples/s]

Filter:   0%|          | 0/11940 [00:00<?, ? examples/s]

In [8]:
dataset[f'test_{id_or_ood}'].filter(lambda row: row['glottocode'] == glottocode and row['is_segmented'] == segmented)

TypeError: '<' not supported between instances of 'NoneType' and 'str'